In [104]:
#Librerias
import numpy as np
import math
import re

In [105]:
#Punto 1 y 2
query1=[1,0,1,0,1,1,1]
query2=[1,1,1,0,1,0,0]
query3=[1,0,1,0,0,0,1,1,1]

In [106]:
#Función para llenar la matriz del query con Revelante
# P@K y R@K
def rPK_RK(arrayQuery):
    mQuery=[]
    relevantes=sum(arrayQuery)
    n=0
    for i in arrayQuery: 
        mQuery.append([])
        mQuery[n].append(i)# Relevante
        if(n==0):
            mQuery[n].append(i) #Relevante inicial acumulativo          
        else:
            mQuery[n].append(i+mQuery[n-1][1]) #Relevante acumulativo
        mQuery[n].append( mQuery[n][1]/relevantes ) # R@K
        mQuery[n].append(mQuery[n][1]/(n+1) ) # P@K
        n+=1
    return mQuery
    
    

In [107]:
#Obtenemos la matriz de cada query (isRelevant,sumRelevant, P@K, R@K)
mQuery1=np.array(rPK_RK(query1))
mQuery2=np.array(rPK_RK(query2))
mQuery3=np.array(rPK_RK(query3))
print("Query 1\n","isRelevant sumRelevant  R@K       P@K ")
print(mQuery1,"\n")
print("Query 2\n","isRelevant sumRelevant  R@K       P@K ")
print(mQuery2,"\n")
print("Query 3\n","isRelevant sumRelevant  R@K       P@K ")
print(mQuery3,"\n")

Query 1
 isRelevant sumRelevant  R@K       P@K 
[[1.         1.         0.2        1.        ]
 [0.         1.         0.2        0.5       ]
 [1.         2.         0.4        0.66666667]
 [0.         2.         0.4        0.5       ]
 [1.         3.         0.6        0.6       ]
 [1.         4.         0.8        0.66666667]
 [1.         5.         1.         0.71428571]] 

Query 2
 isRelevant sumRelevant  R@K       P@K 
[[1.         1.         0.25       1.        ]
 [1.         2.         0.5        1.        ]
 [1.         3.         0.75       1.        ]
 [0.         3.         0.75       0.75      ]
 [1.         4.         1.         0.8       ]
 [0.         4.         1.         0.66666667]
 [0.         4.         1.         0.57142857]] 

Query 3
 isRelevant sumRelevant  R@K       P@K 
[[1.         1.         0.2        1.        ]
 [0.         1.         0.2        0.5       ]
 [1.         2.         0.4        0.66666667]
 [0.         2.         0.4        0.5       ]
 [0.

In [108]:
#Función average Precisión Matrix
def avgP(mQuery):
    n=1
    sumRK=0
    for col in mQuery:
        sumRK+=col[3]
        if(col[2]==1):
            break #Sí P@K = 1 romper        
        n+=1
    return(sumRK/n) 

In [109]:
avgQs=[avgP(mQuery1),avgP(mQuery2),avgP(mQuery3)] 
print("Average Precision Q1: ",avgQs[0])
print("Average Precision Q2: ",avgQs[1])
print("Average Precision Q3: ",avgQs[2]) 
avg_avg=sum(avgQs)/(len(avgQs))
print("\nMAP: ",avg_avg)

Average Precision Q1:  0.6639455782312924
Average Precision Q2:  0.9099999999999999
Average Precision Q3:  0.5426807760141092

MAP:  0.7055421180818006


In [110]:
#Punto 4 y 5
query1_rel=[5,4,4,1,1,1,2]
query2_rel=[4,4,5,1,1,0,2]
query3_rel=[1,1,5,5,2,2,1]

In [111]:

def dfGainDCGI(query_rel):
    mat=[]
    n=0
    for i in query_rel:
        mat.append([])
        mat[n].append(i) #REl_i
        mat[n].append(1/(math.log(max(n+1,2), 2))) #discount_factor
        mat[n].append(i*mat[n][1])
        if(n==0):
            mat[n].append(mat[n][2]) #DCG_i
        else:
            mat[n].append(mat[n][2]+mat[n-1][3]) #DCG_i
        n+=1
    return mat
        

In [112]:
mQueries=[query1_rel,query2_rel,query3_rel]
mMQueries_rel=[]
mMQueries_relN=[] #Normalizada
for i in mQueries:
    mMQueries_rel.append(np.array(dfGainDCGI(i)))
    mMQueries_relN.append(np.array(dfGainDCGI(sorted(i,reverse=True))))

n=len(mMQueries_rel)  
row=(mMQueries_rel[0]).shape[0]
col=(mMQueries_rel[0]).shape[1]

for i in range(n):
    print("Query ",i+1," DCG normalizada",mMQueries_rel[i][row-1][col-1],"") 
    print("Query ",i+1," DCG normalizada",mMQueries_relN[i][row-1][col-1],"\n")  

Query  1  DCG normalizada 13.553662753809808 
Query  1  DCG normalizada 13.697455566701787 

Query  2  DCG normalizada 12.797739700146725 
Query  2  DCG normalizada 13.341248379593765 

Query  3  DCG normalizada 9.645914685581179 
Query  3  DCG normalizada 13.435596059558874 



In [113]:
#Def NDGG
def NDGG(matQ,matQN):
    row=len(matQ)
    col=len(matQ[0])
    return matQ[row-1][col-1]/matQN[row-1][col-1]    

In [114]:
n=len(mMQueries_rel)
sumAvg=0
for i in range(n):
    avg=NDGG(mMQueries_rel[i],mMQueries_relN[i])
    sumAvg+=avg
    print("NDCG Query ",i+1,avg)

print("\nPromedio NDCG ",sumAvg/n)

NDCG Query  1 0.9895022245415027
NDCG Query  2 0.9592610328521903
NDCG Query  3 0.7179372350003413

Promedio NDCG  0.8889001641313449


In [115]:
#Lee los dos documentos de relevancia y el top, luego los une haciendo match 
# de relevancia con los 10 queries del top, y sí hay match en esa posición se asignará 1
# de lo contrario 0 y se almacenará en una matriz

listId=[]
dQuery=dict()
with open("relevance-judgments.tsv", "r") as f:    
    textS=f.read().split("\n")
    for tx in textS:
        if(tx==""):
            break        
        txS=tx.split("\t")
        key=txS[0]
        listId.append(key)
        dQuery[key]=dict()
        txS=txS[1].split(",") 
        for i in range(len(txS)): 
            ts=txS[i].split(":") 
            peso=int(ts[1]) 
            dQuery[key][ts[0]]=peso

MQueries=[]
MQueriesPeso=[]
n=0
with open("TopTenQueries.txt","r") as f:
    textS=f.read().split("\n")
    for tx in textS:
        if(tx==""):
            break
        MQueries.append([])
        MQueriesPeso.append([])
        st=tx.split(":")
        key=st[0]
        stD=st[1].split(";")
        for t in stD:
            if (key in dQuery ) and (t in dQuery[key]):
                MQueries[n].append(1)
                MQueriesPeso[n].append(dQuery[key][t])
            else:
                MQueries[n].append(0)
                MQueriesPeso[n].append(0)
        n+=1 
            

In [116]:
#Procesamiento

mMPKR=[] #Guardo los 331 queries con (isRelevant, relevante, R@K,P@K)
mMQueries_rel2=[] #Guardo los 331 queries con (Rel_i, discount factor, gain, DCG_i)
mMQueries_relN2=[] #Guardo los 331 queries Normalizado con (Rel_i, discount factor, gain, DCG_i)
for i in MQueries:
    mMPKR.append(np.array(rPK_RK(i)))



for i in MQueriesPeso: 
    
    mMQueries_rel2.append(np.array(dfGainDCGI(i))) 
    mMQueries_relN2.append(np.array(dfGainDCGI(sorted(i,reverse=True))))


In [117]:
n=0
prom=[] #Guardo los promedios 
for i in mMPKR:
    av= avgP(i)
    prom.append(av)
    print("Average precision de la ",listId[n],": ", av)
    n+=1

Average precision de la  q01 :  0.5388888888888889
Average precision de la  q02 :  0.16545216663318935
Average precision de la  q03 :  0.042878461749310065
Average precision de la  q04 :  0.14983205419774373
Average precision de la  q06 :  0.3537981298722727
Average precision de la  q07 :  0.07303170630263722
Average precision de la  q08 :  0.23378929833401196
Average precision de la  q09 :  0.1017170732344881
Average precision de la  q10 :  0.10802897248386982
Average precision de la  q12 :  1.0
Average precision de la  q13 :  0.47455357142857135
Average precision de la  q14 :  0.41200665836815636
Average precision de la  q16 :  0.13779781228302576
Average precision de la  q17 :  0.05200951676001193
Average precision de la  q18 :  0.8214285714285714
Average precision de la  q19 :  0.04278117014438744
Average precision de la  q22 :  0.23945246318515268
Average precision de la  q23 :  0.11061670442450386
Average precision de la  q24 :  0.0289942081002992
Average precision de la  q25 :  

In [118]:
#Promedio del avrage precision
print("MAP: ",sum(prom)/(len(prom)))

MAP:  0.29504120427481156


In [119]:
row=(mMQueries_rel2[0]).shape[0]
col=(mMQueries_rel2[0]).shape[1]

 
for i in range(len(mMQueries_rel2)):
    print("DCG Acumulativo de la ",listId[i],": ",mMQueries_rel2[i][row-1][col-1])
    print("DCG Normalizado de la ",listId[i],": ",mMQueries_relN2[i][row-1][col-1])
 

DCG Acumulativo de la  q01 :  9.047411228938167
DCG Normalizado de la  q01 :  12.52371901428583
DCG Acumulativo de la  q02 :  13.57481862850936
DCG Normalizado de la  q02 :  18.984036913935476
DCG Acumulativo de la  q03 :  8.441528358226625
DCG Normalizado de la  q03 :  13.027847991330242
DCG Acumulativo de la  q04 :  9.804072963159074
DCG Normalizado de la  q04 :  11.240262365546286
DCG Acumulativo de la  q06 :  12.826998514234567
DCG Normalizado de la  q06 :  14.027847991330242
DCG Acumulativo de la  q07 :  3.8821681688212064
DCG Normalizado de la  q07 :  8.892789260714373
DCG Acumulativo de la  q08 :  15.89188662298531
DCG Normalizado de la  q08 :  19.580600627037825
DCG Acumulativo de la  q09 :  11.417645596188521
DCG Normalizado de la  q09 :  12.527847991330242
DCG Acumulativo de la  q10 :  8.681228384655356
DCG Normalizado de la  q10 :  14.406929032212952
DCG Acumulativo de la  q12 :  11.52371901428583
DCG Normalizado de la  q12 :  12.02371901428583
DCG Acumulativo de la  q13 :  

In [120]:
#NDGG
aNdgg=[]
for i in range(len(mMQueries_rel2)):
    ndgg=mMQueries_rel2[i][row-1][col-1]/(mMQueries_relN2[i][row-1][col-1])
    aNdgg.append(ndgg)
    print("NDCG de la ",listId[i],": ",ndgg)

print("\nAverage NDCG: ",sum(aNdgg)/len(aNdgg))


NDCG de la  q01 :  0.7224220871306493
NDCG de la  q02 :  0.71506490901019
NDCG de la  q03 :  0.6479603050207743
NDCG de la  q04 :  0.8722281246041527
NDCG de la  q06 :  0.9143953172405456
NDCG de la  q07 :  0.43655236338180636
NDCG de la  q08 :  0.8116138480982569
NDCG de la  q09 :  0.9113812367527109
NDCG de la  q10 :  0.6025731344441758
NDCG de la  q12 :  0.9584155285560207
NDCG de la  q13 :  0.7463424172887285
NDCG de la  q14 :  0.6678693882195809
NDCG de la  q16 :  0.6823387329841737
NDCG de la  q17 :  0.6790302103617731
NDCG de la  q18 :  0.9562932934181377
NDCG de la  q19 :  0.7551681261442821
NDCG de la  q22 :  0.4995683195144241
NDCG de la  q23 :  0.7629714612067607
NDCG de la  q24 :  0.24826585935654275
NDCG de la  q25 :  0.7595077516003776
NDCG de la  q26 :  1.0
NDCG de la  q27 :  0.7200133503065191
NDCG de la  q28 :  0.6746151181807907
NDCG de la  q29 :  0.8830062325362599
NDCG de la  q32 :  0.973635633555801
NDCG de la  q34 :  0.16370565544521562
NDCG de la  q36 :  0.956005

In [121]:
#PUNTO 3 del Taller
#Para el taller de simulador de un motor de búsqueda, calcular P@3, R@3, P@10,
#R@10 (promediado sobre todos los query), asi como el MAP. Crear sus propias
#funciones e incluirlas en el Jupyter Notebook (Suben el Notebook).
p3=[]
r3=[]
p10=[]
r10=[]
for a in mMPKR:
    p3.append(a[2][3])
    r3.append(a[2][2])    
    p10.append(a[9][3])
    r10.append(a[9][2])
    
print("Promedio P@3",sum(p3)/len(p3))
print("Promedio R@3",sum(r3)/len(r3))
print("Promedio P@10",sum(p10)/len(p10))
print("Promedio R@10",sum(p10)/len(p10))


Promedio P@3 0.6095238095238094
Promedio R@3 0.33722531436817155
Promedio P@10 0.35714285714285715
Promedio R@10 0.35714285714285715
